<h3> Aim to build autoencoder trained on only 1 class and measure the reconstruction error for other classes</h3>

In [2]:
#!conda install -y keras
# lstm autoencoder recreate sequence
import pandas as pd
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
#from keras.utils import plot_model
from keras.preprocessing.text import one_hot
import string

# read files and create dataframe
toxicity_comments = pd.read_csv('/mnt/d/Advanced Project/toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
toxicity_annotations = pd.read_csv('/mnt/d/Advanced Project/toxicity_annotations.tsv',  sep = '\t')
toxicity_demographics = pd.read_csv('/mnt/d/Advanced Project/toxicity_worker_demographics.tsv', sep = '\t')

toxicity = toxicity_comments.merge(toxicity_annotations, how ='outer', on="rev_id")
toxicity = toxicity.merge(toxicity_demographics, how ='outer', on="worker_id").sort_values(by=['rev_id','worker_id'])

# remove newline and tab tokens
toxicity['comment'] = toxicity['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
toxicity['comment'] = toxicity['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

# add binary gender columns
toxicity = toxicity[toxicity['gender']!='other']
toxicity = pd.concat([toxicity, pd.get_dummies(toxicity.gender).rename(columns = "{}_binary".format)], axis = 1)

# limit size of dataset for testing purposes
very_toxic = toxicity[toxicity.toxicity_score == -2.0]
female_vtoxic = very_toxic[very_toxic.female_binary == 1]

female_vtoxic['comment'] = female_vtoxic['comment'].str.translate(str.maketrans('', '', string.punctuation))
female_vtoxic['comment'] = female_vtoxic['comment'].replace('\n',' ', regex=True)
female_vtoxic['comment'] = female_vtoxic['comment'].str.lower()
# female_vtoxic['comment'] = female_vtoxic['comment'].str.split() 
#display(female_vtoxic['comment'].head(5))

comments = female_vtoxic.comment.values
# testing on smaller set 
comments = comments[:100]
scores = female_vtoxic.toxicity_score.values
# for i in range(len(comments)):
#   cmt = str(scores[i])+" "+comments[i]
#   comments[i] = cmt
print(comments[:5])

['  oh never mind  i misread the change  my mistake   egern  '
 '    erik for crying out loud you legally can have sex with children but not with underage children underage children are children beneath the legal age of consent different states have 14 15 16 17 or 18 as the underage limit children in most countries means someone under the age of 18 in ireland for example a child aged 17 is over the age of consent in britain it is 16 do you know the first thing about the topic   the name is standard english asage scandal is a pov term allegation allows the discussion of cases that have not yet come to court which cannot for technical reasons be prosecuted and cases that were thrown up as invalid roman catholic church sex abuses allegations in standard english means allegations about the roman catholic church   public relations is minor tabloid point when you are discussing the rape of children so stop adding in an irrelevant topic   your mucking around lost a lot of changes spelling cor

<ipython-input-2-708a932df6d7>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_vtoxic['comment'] = female_vtoxic['comment'].str.translate(str.maketrans('', '', string.punctuation))
<ipython-input-2-708a932df6d7>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_vtoxic['comment'] = female_vtoxic['comment'].replace('\n',' ', regex=True)
<ipython-input-2-708a932df6d7>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [5]:
from keras.preprocessing.sequence import pad_sequences
# integer encode the documents
vocab_size =10
encoded_docs = [one_hot(c, vocab_size) for c in comments]
# pad documents to a max length of 4 words
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs[:5])

from keras.preprocessing.text import Tokenizer
num_words = 2000
maxlen = 30
embed_dim = 150
batch_size = 16
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(comments)
seqs = tokenizer.texts_to_sequences(comments)
pad_seqs = pad_sequences(seqs, maxlen)
print(pad_seqs[:2])

[[1 1 1 2 3 9 4 5 3 2 0 0 0 0 0 0 0 0 0 0]
 [6 2 9 2 8 1 2 2 4 2 3 9 3 3 1 2 4 5 9 4]
 [9 6 9 2 2 9 8 6 7 5 8 2 6 7 9 6 9 5 8 8]
 [5 9 4 1 4 1 1 9 3 7 1 2 8 4 9 8 1 7 9 2]
 [7 7 4 4 4 6 4 4 8 7 4 2 4 4 0 0 0 0 0 0]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 232 233 324   5 855   1 325  35 326 856]
 [103 340 338  27  60   2 487  11 895  53 241  24   1 488 896 145   7 334
  245   5  29 146 897 164   3   7 898 899 900 245]]


In [4]:
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras import Input
from keras import Model
from keras import regularizers
from keras.layers import Bidirectional
from keras import optimizers

encoder_inputs = Input(shape=(maxlen,), name='Encoder-Input')
emb_layer = Embedding(num_words, embed_dim,input_length = maxlen, name='Body-Word-Embedding', mask_zero=False)
x = emb_layer(encoder_inputs)
state_h = Bidirectional(LSTM(128, activation='relu', name='Encoder-Last-LSTM'))(x)
encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')
seq2seq_encoder_out = encoder_model(encoder_inputs)

decoded = RepeatVector(maxlen)(seq2seq_encoder_out)
decoder_lstm = Bidirectional(LSTM(128, return_sequences=True, name='Decoder-LSTM-before'))
decoder_lstm_output = decoder_lstm(decoded)
decoder_dense = Dense(num_words, activation='softmax', name='Final-Output-Dense-before')
decoder_outputs = decoder_dense(decoder_lstm_output)

seq2seq_Model = Model(encoder_inputs, decoder_outputs)
seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')
history = seq2seq_Model.fit(pad_seqs, np.expand_dims(pad_seqs, -1),
          batch_size=batch_size,
          epochs=10)

y = history.predict(pad_seqs)
print(y)

sentence = "here's a sample unseen sentence"
seq = tokenizer.texts_to_sequences([sentence])
pad_seq = pad_sequences(seq, maxlen)
sentence_vec = encoder_model.predict(pad_seq)[0]
print(sentence_vec)

# input_layer = Input(shape=max_length)

# ## encoding part
# encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
# encoded = Dense(50, activation='relu')(encoded)

# ## decoding part
# decoded = Dense(50, activation='tanh')(encoded)
# decoded = Dense(100, activation='tanh')(decoded)

# ## output layer
# output_layer = Dense(max_length, activation='relu')(decoded)

# autoencoder = Model(input_layer, output_layer)
# autoencoder.compile(optimizer="adam", loss="mse")

# autoencoder.fit(padded_docs, padded_docs, 
#                 batch_size = 256, epochs = 10, 
#                 shuffle = True, validation_split = 0.20);

# # hidden_representation = Sequential()
# # hidden_representation.add(autoencoder.layers[0])
# # hidden_representation.add(autoencoder.layers[1])
# # hidden_representation.add(autoencoder.layers[2])
# y = autoencoder.predict(padded_docs)
# print(y)
# print(len(y))
# print(len(y[0]))



# define input sequence
#sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
#sequence = [[[i] for i in comments[0]]]
#n_in = len(sequence)
#sequence = sequence.reshape((1, n_in, 1))
#print(sequence)
# #define model
# model = Sequential()
# model.add(Embedding(vocab_size, 8, input_length=max_length))
# model.add(LSTM(100, activation='relu'))
# model.add(RepeatVector(max_length))
# model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(TimeDistributed(Dense(1)))
# model.compile(optimizer='adam', loss='categorical_crossentropy')
# # fit model
# model.fit(padded_docs, padded_docs, epochs=100, verbose=0)
# #plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# # demonstrate recreation
# yhat = model.predict([padded_docs[0]], verbose=0)
# print(yhat)
# print(yhat[0,:,0])

# model = Sequential()
# model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
# model.add(RepeatVector(n_in))
# model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(TimeDistributed(Dense(1)))
# model.compile(optimizer='adam', loss='mse')
# # fit model
# model.fit(sequence, sequence, epochs=300, verbose=0)
# plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# # demonstrate recreation
# yhat = model.predict(sequence, verbose=0)
# print(yhat[0,:,0])

# #model for text summarisation
# inputs = Input(shape=(max_length))
# encoder1 = Embedding(vocab_size, 128)(inputs)
# encoder2 = LSTM(128)(encoder1)
# encoder3 = RepeatVector(max_length)(encoder2)
# # decoder output model
# decoder1 = LSTM(128, return_sequences=True)(encoder3)
# outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# # tie it together
# model = Model(inputs=inputs, outputs=outputs)
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# print(comments[0])
# print(padded_docs[0])
# yhat = model.predict(padded_docs[0], verbose=0)
# print(yhat[0,:,0])

# # define the model
# model = Sequential()
# model.add(Embedding(vocab_size, 8, input_length=max_length))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# # compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy')
# # summarize the model
# print(model.summary())
# # fit the model
# print(padded_docs.shape)
# model.fit(padded_docs,padded_docs, epochs=50, verbose=0)
# yhat = model.predict(padded_docs[0], verbose=0)
# print(yhat)



Epoch 1/10
7/7 [==============================] - 1s 181ms/step - loss: 7.5454
Epoch 2/10
7/7 [==============================] - 1s 180ms/step - loss: 5.6540
Epoch 3/10
7/7 [==============================] - 1s 164ms/step - loss: 5.1384
Epoch 4/10
7/7 [==============================] - 1s 168ms/step - loss: 5.0348
Epoch 5/10
7/7 [==============================] - 1s 124ms/step - loss: 5.0233
Epoch 6/10
7/7 [==============================] - 1s 213ms/step - loss: 4.9878
Epoch 7/10
7/7 [==============================] - 1s 126ms/step - loss: 4.9388
Epoch 8/10
7/7 [==============================] - 1s 128ms/step - loss: 4.9584
Epoch 9/10
7/7 [==============================] - 1s 173ms/step - loss: 4.8649
Epoch 10/10
7/7 [==============================] - 1s 190ms/step - loss: 4.9030


AttributeError: 'History' object has no attribute 'predict'